In [1]:
import numpy as np
import glob
import json
import csv
from gufe.tokenization import JSON_HANDLER

In [2]:
def load_results(f):
    # path to deserialized results
    with open(f, "r") as fd:
        result = json.load(fd, cls=JSON_HANDLER.decoder)
    return result

In [3]:
def extract(results_0, results_1, results_2, output):
    files_0 = glob.glob(f"{results_0}/*.json")
    files_1 = glob.glob(f"{results_1}/*.json")
    files_2 = glob.glob(f"{results_2}/*.json")
    with open(output, 'w', newline='') as csvfile:
        writer = csv.writer(
            csvfile,
            delimiter="\t",
            lineterminator="\n",  # to exactly reproduce previous, prefer "\r\n"
        )
        writer.writerow(["ligand", "DG(i->j) (kcal/mol)",
                         "uncertainty (kcal/mol)"])
        for file in files_0:
            json_0 = load_results(file)
            dg_0 = json_0["estimate"].magnitude
            try:
                file_1 = f"{results_1}/{file.split("/")[-1]}"
                file_2 = f"{results_2}/{file.split("/")[-1]}"
                json_1 = load_results(file_1)
                json_2 = load_results(file_2)
                dg_1 = json_1["estimate"].magnitude
                dg_2 = json_2["estimate"].magnitude
                dgs = [dg_0, dg_1, dg_2]
                dg = np.mean(dgs)
                dg_error = np.std(dgs)
                writer.writerow([file.split('/')[-1].split('.')[0], round(dg, 2), round(dg_error, 2)])
            except FileNotFoundError:
                continue

In [ ]:
extract('results_0', 'results_1', 'results_2', 'ddg.tsv')

/Users/hannahbaumann/miniforge3/envs/septop/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(
